In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(42)

This sets the no. of data points we make.

In [3]:
num_samples = 1000

Generating the dataset using Numpy

np.random.normal is used to generate random data points with a mean value and a standard deviation value

In [4]:
payload = np.random.normal(loc=145.205, scale=17.082, size=num_samples)

round_trip_distance = np.random.normal(loc=5, scale=1, size=num_samples)  

cycles_per_shift = np.random.normal(loc=1, scale=1, size=num_samples)  

total_hours_operation = np.random.normal(loc=8, scale=1, size=num_samples) 

awss = (round_trip_distance * cycles_per_shift) / total_hours_operation

temperature = np.random.normal(loc=30, scale=5, size=num_samples)  

tire_pressure = np.random.normal(loc=100, scale=10, size=num_samples)

The ideal air pressure for a dumper truck tire depends on the truck's weight and other factors, but generally ranges from 8 to 9 bar (116–131 PSI). However, most truckers tend to inflate their tires to between 85 and 105 PSI, with 95 PSI being the most common

In [5]:
anomalies = np.random.choice([0, 1], size=num_samples, p=[0.90, 0.10])  
tire_pressure = np.where(anomalies == 1, np.random.choice([np.random.uniform(60, 85), np.random.uniform(110, 140)], size=num_samples), tire_pressure)

tire_pressure = np.clip(tire_pressure, 60, 140)

anomaly = np.where((tire_pressure < 85) | (tire_pressure > 110), 1, 0)

In [6]:
df = pd.DataFrame({
    'Payload (Ton)': payload,
    'Round-trip Distance (km)': round_trip_distance,
    'Cycles/Shift': cycles_per_shift,
    'Total Hours Operation/Shift': total_hours_operation,
    'AWSS': awss,
    'Tire Pressure (psi)': tire_pressure,
    'Temperature (°C)': temperature,
    'Anomaly': anomaly
})


### Dataset Generated

In [7]:
df.head()

,Payload (Ton),Round-trip Distance (km),Cycles/Shift,Total Hours Operation/Shift,AWSS,Tire Pressure (psi),Temperature (°C),Anomaly
0,153.689871,6.399355,0.324822,6.092192,0.341199,95.762403,25.682532,0
1,142.843169,5.924634,0.855481,7.139615,0.709900,95.465859,29.843983,0
2,156.268816,5.059630,0.207580,7.586394,0.138442,82.043568,30.090084,1
3,171.221396,4.353063,0.692038,9.887688,0.304671,96.699098,32.363152,0
4,141.205192,5.698223,-0.893615,8.556553,-0.595101,76.006273,23.165708,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Payload (Ton)                1000 non-null   float64
 1   Round-trip Distance (km)     1000 non-null   float64
 2   Cycles/Shift                 1000 non-null   float64
 3   Total Hours Operation/Shift  1000 non-null   float64
 4   AWSS                         1000 non-null   float64
 5   Tire Pressure (psi)          1000 non-null   float64
 6   Temperature (°C)             1000 non-null   float64
 7   Anomaly                      1000 non-null   int32  
dtypes: float64(7), int32(1)
memory usage: 58.7 KB


Saving the dataset

In [9]:
df.to_csv('anomaly_detection.csv', index=False)

In [10]:
X = df[['Payload (Ton)', 'AWSS', 'Tire Pressure (psi)', 'Temperature (°C)']]
y = df['Anomaly']

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Making the model in Tensorflow

In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


# Create a simple neural network that approximates the decision tree
model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(len(set(y_train)), activation='softmax')  # Assuming a classification task
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Save the TensorFlow model
model.save('tf_model.h5')




Epoch 1/50


20/20 [==============================] - 4s 36ms/step - loss: 8.7812 - accuracy: 0.5203 - val_loss: 3.6716 - val_accuracy: 0.7812
Epoch 2/50
20/20 [==============================] - 0s 11ms/step - loss: 3.6534 - accuracy: 0.6125 - val_loss: 2.4561 - val_accuracy: 0.5938
Epoch 3/50
20/20 [==============================] - 0s 9ms/step - loss: 2.6260 - accuracy: 0.6766 - val_loss: 1.8861 - val_accuracy: 0.6062
Epoch 4/50
20/20 [==============================] - 0s 9ms/step - loss: 2.0930 - accuracy: 0.6234 - val_loss: 1.3746 - val_accuracy: 0.5625
Epoch 5/50
20/20 [==============================] - 0s 9ms/step - loss: 1.3415 - accuracy: 0.6328 - val_loss: 0.7826 - val_accuracy: 0.7437
Epoch 6/50
20/20 [==============================] - 0s 10ms/step - loss: 0.8192 - accuracy: 0.6141 - val_loss: 0.5457 - val_accuracy: 0.7563
Epoch 7/50
20/20 [==============================] - 0s 9ms/step - loss: 0.6704 - accuracy: 0.6625 - val_loss: 0.5124 - val_accuracy: 0.7812
Epoch 8/50
2

C:\Users\RYZEN-5\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### Test Accuracy of 81%

Converting the saved model into .tflite format so that it can be used in the app

In [13]:
# Load the saved TensorFlow model
loaded_model = tf.keras.models.load_model('tf_model.h5')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\RYZEN-5\AppData\Local\Temp\tmpmdcfjfq0\assets


INFO:tensorflow:Assets written to: C:\Users\RYZEN-5\AppData\Local\Temp\tmpmdcfjfq0\assets


Sample Function to check whether the model works 

In [14]:
import numpy as np

def detect_anomaly(payload, awss, tire_pressure, temperature, model):
    input_data = np.array([[payload, awss, tire_pressure, temperature]])
    prediction = model.predict(input_data)
    
    # For binary classification (e.g., sigmoid activation)
    if prediction.shape[1] == 1:
        confidence = prediction[0][0]
        is_anomaly = confidence > 0.5
    else:  # For multi-class classification (e.g., softmax activation)
        confidence = np.max(prediction[0])
        is_anomaly = np.argmax(prediction[0]) == 1
    
    return is_anomaly, confidence


In [22]:
is_anomaly, confidence = detect_anomaly(173.68, 0.34, 77.76, 25.68, model)
print(f"Anomaly Detected: {is_anomaly}, Confidence: {confidence:.2f}")

1/1 [==============================] - 0s 102ms/step
Anomaly Detected: True, Confidence: 0.68
